In [1]:
import torch
import gym

import numpy as np

# from ji_dog_net_v1 import PPO_Clip
# from ji_dog_net_v1 import ActorCritic_Clip
# from ji_dog_net_v1 import process_state

from ji_dog_net_v2 import PPO_Penalty
from ji_dog_net_v2 import ActorCritic_Penalty

from ji_dog_net_v3 import PPO_Clip
from ji_dog_net_v3 import ActorCritic_Clip
from ji_dog_net_v3 import process_state


## Check device

In [2]:
print("============================================================================================")
# set device to cpu or cuda
device = torch.device('cpu')
if(torch.cuda.is_available()): 
    device = torch.device('cuda:0') 
    torch.cuda.empty_cache()
    print("Device set to : " + str(torch.cuda.get_device_name(device)))
else:
    print("Device set to : cpu")
print("============================================================================================")


Device set to : NVIDIA GeForce RTX 4060 Laptop GPU


In [3]:
from isaacsim import SimulationApp
simulation_app = SimulationApp({"headless": False})

from torch.utils.tensorboard import SummaryWriter

from ji_dog_env_create_v3 import Ji_Dog_Env
from tqdm import tqdm  
import numpy as np
import os

class PIDController:
    def __init__(self, kp, ki, kd, output_limits=(None, None)):
        self.kp = kp
        self.ki = ki
        self.kd = kd
        self.prev_error = 0
        self.integral = 0
        self.min_output, self.max_output = output_limits

    def compute(self, target, current):
        error = target - current
        self.integral += error
        derivative = error - self.prev_error
        self.prev_error = error

        output = self.kp * error + self.ki * self.integral + self.kd * derivative

        if self.min_output is not None:
            output = max(self.min_output, output)
        if self.max_output is not None:
            output = min(self.max_output, output)

        return output




state_dim = 20
action_dim = 4
max_training_timesteps = 100
max_timesteps = 5000


pid_controllers = [PIDController(kp=1.0, ki=0.1, kd=0.5, output_limits=(-1, 1)) for _ in range(action_dim)]

target_values = np.zeros(action_dim)  

env = Ji_Dog_Env('Model(including video)/ji_dog1.0.usd')



state = env.reset()
state = process_state(state)
state = np.array(state)

total_episode_reward = 0

for t in range(max_timesteps + 1):

    current_values = state[:action_dim] 
    action = np.array([pid.compute(target, current) for pid, target, current in zip(pid_controllers, target_values, current_values)])
    
    state, total_reward, rewards, done, _ = env.step(action)

    state = process_state(state)
    state = np.array(state)

    total_episode_reward += total_reward
    print(action)
    print(f"Step {t}: Reward = {total_reward:.2f}, Total Reward = {total_episode_reward:.2f}")

    if done:
        break


Starting kit application with the following args:  ['/home/bai/.local/share/ov/pkg/isaac-sim-4.2.0/exts/omni.isaac.kit/omni/isaac/kit/simulation_app.py', '/home/bai/.local/share/ov/pkg/isaac-sim-4.2.0/apps/omni.isaac.sim.python.kit', '--/app/tokens/exe-path=/home/bai/.local/share/ov/pkg/isaac-sim-4.2.0/kit', '--/persistent/app/viewport/displayOptions=3094', '--/rtx/materialDb/syncLoads=True', '--/rtx/hydra/materialSyncLoads=True', '--/omni.kit.plugin/syncUsdLoads=True', '--/app/renderer/resolution/width=1280', '--/app/renderer/resolution/height=720', '--/app/window/width=1440', '--/app/window/height=900', '--/renderer/multiGpu/enabled=True', '--/app/fastShutdown=False', '--ext-folder', '/home/bai/.local/share/ov/pkg/isaac-sim-4.2.0/exts', '--ext-folder', '/home/bai/.local/share/ov/pkg/isaac-sim-4.2.0/apps', '--/physics/cudaDevice=0', '--portable']
Passing the following args to the base kit application:  ['-f', '/home/bai/.local/share/jupyter/runtime/kernel-6bdffea4-da76-44e5-8c33-50c5b

In [ ]:
simulation_app.close()
